# Run PCA and create the svg with the plot  and create the fastSTRUCTURE input file

Create an file list with all the vcf:

In [2]:
# Create list of all VCF prefixes
!ls 01_vcfs/*.vcf | sed 's|01_vcfs/||' | sed 's|.vcf||' > vcf_prefixes.txt

# Check the content
!cat vcf_prefixes.txt

alti_50
alti_80
alti_90
alti_95
grp_50
grp_80
grp_90
grp_95
phylo_50
phylo_80
phylo_90
phylo_95


The RScript:

In [ ]:
#!/usr/bin/env Rscript 
# Load required packages
library(vcfR)
library(adegenet)
library(ggplot2)
library(dartR)

# Capture command line arguments
args <- commandArgs(trailingOnly = TRUE)
if (length(args) < 1) {
  stop("Usage: Rscript script.R <file_with_prefix_list>")
}

# The first argument is the file with the list of prefixes
prefix_list_file <- args[1]

# Read prefixes from file
prefixes <- readLines(prefix_list_file)

# Loop to process each prefix
for (prefix in prefixes) {
  
  tryCatch({
    # Define file paths
    vcf_file <- paste0("01_vcfs/", prefix, ".vcf")
    
    # Check if VCF file exists
    if (!file.exists(vcf_file)) {
      cat("❌ VCF file not found:", vcf_file, "\n")
      next
    }
    
    cat("📊 Processing:", prefix, "\n")
    
    # Extract group name from prefix
    group_name <- sub("_.*", "", prefix)
    
    # Define popmap file
    popmap_file <- paste0("popmap_", group_name)
    
    # Check if popmap file exists
    if (!file.exists(popmap_file)) {
      cat("❌ Popmap file not found:", popmap_file, "\n")
      next
    }
    
    # Define output file
    output_svg <- paste0("02_PCA/",prefix, "_pca.svg")
    output_data <- paste0("02_PCA/",prefix, "_pca_data.txt")

    # Read VCF file
    vcf <- read.vcfR(vcf_file)
    vcf_gl <- vcfR2genlight(vcf)
    
    # Read popmap
    popmap_data <- read.table(popmap_file, header = FALSE, sep = "\t", 
                             stringsAsFactors = FALSE)
    colnames(popmap_data) <- c("sample", "population")
    
    # Match samples with popmap
    sample_names <- indNames(vcf_gl)
    pop_vector <- character(length(sample_names))
    
    for (i in 1:length(sample_names)) {
      sample_match <- which(popmap_data$sample == sample_names[i])
      if (length(sample_match) > 0) {
        pop_vector[i] <- popmap_data$population[sample_match[1]]
      } else {
        pop_vector[i] <- "Unknown"
        cat("⚠️  Sample not found in popmap:", sample_names[i], "\n")
      }
    }
    
    # Set populations
    pop(vcf_gl) <- pop_vector
    
    ## Perform PCA
    # nf = an integer indicating the number of principal components to be retained; 
    # if NULL, a screeplot of eigenvalues will be displayed and 
    # the user will be asked for a number of retained axes.
    # pca1 <- glPca(vcf_gl, nf = 2)
    
    # # Save PCA data
    write.table(pca1$scores, file = output_data, sep = "\t", quote = FALSE)
    
    # Generate PCA plot
    svg(output_svg, width = 10, height = 8)
    gl.pcoa.plot(pca1, vcf_gl, pop.labels = "pop", xaxis = 1, yaxis = 2, 
                 pt.size = 6, label.size = 2)
    dev.off()
    
    cat(" Completed:", prefix, "\n")
    cat("   - Samples:", length(sample_names), "\n")
    cat("   - Populations:", length(unique(pop_vector)), "\n")
    cat("   - Output:", output_svg, "\n")

    # fast STRUTURE input
    output_str <- paste0(prefix,".str")
    outpath <- paste0("03_fastSTRUCTURE/")

    gl2faststructure(vcf_gl, outfile = output_str, outpath = outpath)
  }, error = function(e) {
    cat("Error processing", prefix, ":", e$message, "\n")
  }
)
}

cat("All processing completed!\n")

Run:

In [ ]:
!Rscript Script-run-PCA.R vcf_prefixes.txt  > PCA-run.log 2>&1


   *****       ***   vcfR   ***       *****
   This is vcfR 1.15.0 
     browseVignettes('vcfR') # Documentation
     citation('vcfR') # Citation
   *****       *****      *****       *****

Loading required package: ade4

   /// adegenet 2.1.11 is loaded ////////////

   > overview: '?adegenet'
   > tutorials/doc/questions: 'adegenetWeb()' 
   > bug reports/feature requests: adegenetIssues()


Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: dartR.data
**** Welcome to dartR.data [Version 1.0.8 ] ****

Registered S3 method overwritten by 'pegas':
  method      from
  print.amova ade4
Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2
Registered S3 method overwritten by 'genetics':
  method      from 
  [.haplotype pegas
**** Welcome to dartR [Version 2.

I run in the terminal to do just the input because I added the part of the -str only after

# Run fastSTRUCTURE

Run script on byobu: ```Script-Run-fastStructure.py```

In [ ]:
import os
import subprocess
import datetime

# Create output folder
output_folder = "03_fastSTRUCTURE"
os.makedirs(output_folder, exist_ok=True)

# Create log file
log_file = f"faststructure_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

# Read prefixes
with open('vcf_prefixes.txt', 'r') as f:
    prefixes = [line.strip() for line in f.readlines()]

print(f"Found {len(prefixes)} prefixes to process")
print(f"Log file: {log_file}")

# Open log file for writing
with open(log_file, 'w') as log:
    log.write(f"Started processing at {datetime.datetime.now()}\n")
    log.write(f"Number of prefixes: {len(prefixes)}\n\n")
    
    success_count = 0
    fail_count = 0
    
    for prefix in prefixes:
        group_name = prefix.split('_')[0]
        str_file = f"03_fastSTRUCTURE/{prefix}.str"
        popmap_file = f"popmap_{group_name}"
        output_prefix = f"03_fastSTRUCTURE/result_{prefix}"
        
        # Check if files exist
        if not os.path.exists(str_file):
            msg = f"ERROR: STR file not found: {str_file}"
            print(msg)
            log.write(msg + '\n')
            fail_count += 1
            continue
        
        if not os.path.exists(popmap_file):
            msg = f"ERROR: Popmap file not found: {popmap_file}"
            print(msg)
            log.write(msg + '\n')
            fail_count += 1
            continue
        
        msg = f"PROCESSING: {prefix}"
        print(msg)
        log.write(msg + '\n')
        
        try:
            # Run structure_threader
            result = subprocess.run([
                "structure_threader", "run", 
                "-K", "10", "-R", "10",
                "-i", str_file, 
                "-o", output_prefix, 
                "-t", "16",
                "-fs", os.path.expanduser("~/.local/bin/fastStructure"),
                "--pop", popmap_file
            ], capture_output=True, text=True, check=True)
            
            # Log success
            msg = f"SUCCESS: {prefix}"
            print(msg)
            log.write(msg + '\n')
            
            # Log any output from the command
            if result.stdout:
                log.write("Command output:\n")
                log.write(result.stdout + '\n')
                
            success_count += 1
            
        except subprocess.CalledProcessError as e:
            msg = f"FAILED: {prefix}"
            print(msg)
            log.write(msg + '\n')
            log.write(f"Error: {e.stderr}\n")
            fail_count += 1
            
        except Exception as e:
            msg = f"UNEXPECTED ERROR: {prefix} - {e}"
            print(msg)
            log.write(msg + '\n')
            fail_count += 1
        
        log.write('\n')  # Empty line between entries
    
    # Write summary
    log.write("\n=== SUMMARY ===\n")
    log.write(f"Successful: {success_count}\n")
    log.write(f"Failed: {fail_count}\n")
    log.write(f"Completed at: {datetime.datetime.now()}\n")

print(f"Processing completed! Check {log_file} for details.")
print(f"Summary: {success_count} successful, {fail_count} failed")

log:
<br>
>Started processing at 2025-10-08 14:50:27.416532
>Number of prefixes: 12
>
>PROCESSING: alti_50
>SUCCESS: alti_50
>
>PROCESSING: alti_80
>SUCCESS: alti_80
>
>PROCESSING: alti_90
>SUCCESS: alti_90
>
>PROCESSING: alti_95
>SUCCESS: alti_95
>
>PROCESSING: grp_50
>SUCCESS: grp_50
>
>PROCESSING: grp_80
>SUCCESS: grp_80
>
>PROCESSING: grp_90
>SUCCESS: grp_90
>
>PROCESSING: grp_95
>SUCCESS: grp_95
>
>PROCESSING: phylo_50
>SUCCESS: phylo_50
>
>PROCESSING: phylo_80
>SUCCESS: phylo_80
>
>PROCESSING: phylo_90
>SUCCESS: phylo_90
>
>PROCESSING: phylo_95
>SUCCESS: phylo_95
>
>
>=== SUMMARY ===
>Successful: 12
>Failed: 0
>Completed at: 2025-10-08 14:55:45.639421
<br>

## Get result fastSTRUCUTRE

In [1]:
import os
import pandas as pd
import re
from pathlib import Path

In [2]:
def extract_best_k(choosek_file):
    """Extract K values from choosek.txt file"""
    if not os.path.exists(choosek_file):
        return None, None
    
    try:
        with open(choosek_file, 'r') as f:
            content = f.read()
        
        # Patterns to extract numbers
        marginal_pattern = r'Model complexity that maximizes marginal likelihood = (\d+)'
        components_pattern = r'Model components used to explain structure in data = (\d+)'
        
        marginal_match = re.search(marginal_pattern, content)
        components_match = re.search(components_pattern, content)
        
        if marginal_match and components_match:
            marginal_k = int(marginal_match.group(1))
            components_k = int(components_match.group(1))
            return marginal_k, components_k
        else:
            return None, None
            
    except Exception as e:
        print(f"Error reading {choosek_file}: {e}")
        return None, None

In [6]:
print("🔍 Searching for fastSTRUCTURE results...")

base_folder = "03_fastSTRUCTURE"
results = []

# List all result folders
for item in os.listdir(base_folder):
    if item.startswith('result_'):
        prefix = item.replace('result_', '')
        choosek_path = os.path.join(base_folder, item, 'bestK', 'chooseK.txt')
        
        marginal_k, components_k = extract_best_k(choosek_path)
        
        if marginal_k is not None:
            # Extract group and parameter from prefix
            if '_' in prefix:
                group = prefix.split('_')[0]
                parameter = prefix.split('_')[1]
            else:
                group = prefix
                parameter = "unknown"
            
            results.append({
                'Group': group,
                'Parameter': parameter,
                'Prefix': prefix,
                'Marginal_K': marginal_k,
                'Components_K': components_k
            })
            print(f"✅ Extracted: {prefix} → Marginal K: {marginal_k}, Components K: {components_k}")
        else:
            print(f"❌ No results found for: {prefix}")

🔍 Searching for fastSTRUCTURE results...
✅ Extracted: alti_50 → Marginal K: 3, Components K: 5
✅ Extracted: alti_95 → Marginal K: 3, Components K: 6
✅ Extracted: grp_80 → Marginal K: 5, Components K: 5
✅ Extracted: phylo_90 → Marginal K: 4, Components K: 5
✅ Extracted: grp_95 → Marginal K: 2, Components K: 4
✅ Extracted: alti_90 → Marginal K: 3, Components K: 4
✅ Extracted: phylo_80 → Marginal K: 4, Components K: 5
✅ Extracted: phylo_95 → Marginal K: 4, Components K: 5
✅ Extracted: alti_80 → Marginal K: 3, Components K: 5
✅ Extracted: grp_90 → Marginal K: 5, Components K: 4
✅ Extracted: grp_50 → Marginal K: 7, Components K: 5
✅ Extracted: phylo_50 → Marginal K: 4, Components K: 4


In [9]:
results

[{'Group': 'alti',
  'Parameter': '50',
  'Prefix': 'alti_50',
  'Marginal_K': 3,
  'Components_K': 5},
 {'Group': 'alti',
  'Parameter': '95',
  'Prefix': 'alti_95',
  'Marginal_K': 3,
  'Components_K': 6},
 {'Group': 'grp',
  'Parameter': '80',
  'Prefix': 'grp_80',
  'Marginal_K': 5,
  'Components_K': 5},
 {'Group': 'phylo',
  'Parameter': '90',
  'Prefix': 'phylo_90',
  'Marginal_K': 4,
  'Components_K': 5},
 {'Group': 'grp',
  'Parameter': '95',
  'Prefix': 'grp_95',
  'Marginal_K': 2,
  'Components_K': 4},
 {'Group': 'alti',
  'Parameter': '90',
  'Prefix': 'alti_90',
  'Marginal_K': 3,
  'Components_K': 4},
 {'Group': 'phylo',
  'Parameter': '80',
  'Prefix': 'phylo_80',
  'Marginal_K': 4,
  'Components_K': 5},
 {'Group': 'phylo',
  'Parameter': '95',
  'Prefix': 'phylo_95',
  'Marginal_K': 4,
  'Components_K': 5},
 {'Group': 'alti',
  'Parameter': '80',
  'Prefix': 'alti_80',
  'Marginal_K': 3,
  'Components_K': 5},
 {'Group': 'grp',
  'Parameter': '90',
  'Prefix': 'grp_90',
 

In [11]:
if results:
    df = pd.DataFrame(results)
    
    # Sort by Group and Parameter
    df['Parameter_Num'] = pd.to_numeric(df['Parameter'], errors='coerce')
    df = df.sort_values(['Group', 'Parameter_Num']).drop('Parameter_Num', axis=1)
    
    # Display table
    print("\n RESULTS TABLE:")
    print("=" * 70)
    display(df)
    print("=" * 70)
    
    # Save to CSV
    csv_file = "03_fastSTRUCTURE/faststructure_results.csv"
    df.to_csv(csv_file, index=False)
    print(f"\n Results saved to: {csv_file}")
    
    # Summary statistics
    print("\n STATISTICS BY GROUP:")
    summary = df.groupby('Group').agg({
        'Marginal_K': ['count', 'mean', 'min', 'max'],
        'Components_K': ['mean', 'min', 'max']
    }).round(2)
    
    display(summary)
    
else:
    print("❌ No results found. Please check if the folders exist.")


 RESULTS TABLE:


,Group,Parameter,Prefix,Marginal_K,Components_K
0,alti,50,alti_50,3,5
8,alti,80,alti_80,3,5
5,alti,90,alti_90,3,4
1,alti,95,alti_95,3,6
10,grp,50,grp_50,7,5
2,grp,80,grp_80,5,5
9,grp,90,grp_90,5,4
4,grp,95,grp_95,2,4
11,phylo,50,phylo_50,4,4
6,phylo,80,phylo_80,4,5



 Results saved to: 03_fastSTRUCTURE/faststructure_results.csv

 STATISTICS BY GROUP:


Marginal_K               Components_K        
           count  mean min max         mean min max
Group                                              
alti           4  3.00   3   3         5.00   4   6
grp            4  4.75   2   7         4.50   4   5
phylo          4  4.00   4   4         4.75   4   5

In [17]:
import os
import shutil

def simple_folder_based_copy(k_assignments):
    """
    Simple version that only uses folder names
    """
    base_folder = "03_fastSTRUCTURE"
    output_folder = "3_fastSTRUCTURE/BEST_results"
    os.makedirs(output_folder, exist_ok=True)
    
    print("🎯 Copying .meanQ files using folder names...")
    
    success_count = 0
    
    for item in os.listdir(base_folder):
        if item.startswith('result_'):
            prefix = item.replace('result_', '')
            
            if '_' in prefix:
                group = prefix.split('_')[0]
                parameter = prefix.split('_')[1]
                
                # Get specified K for this group
                specified_k = k_assignments.get(group)
                
                if specified_k:
                    source_folder = os.path.join(base_folder, item)
                    
                    # Look for the .meanQ file
                    found = False
                    if os.path.exists(source_folder):
                        for file in os.listdir(source_folder):
                            if file.endswith('.meanQ'):
                                # Check if this file matches our specified K
                                if (f'fS_run_K.{specified_k}.' in file):
                                    
                                    new_name = f"{group}_{parameter}_k{specified_k}.meanQ"
                                    destination = os.path.join(output_folder, new_name)
                                    shutil.copy2(os.path.join(source_folder, file), destination)
                                    print(f"✅ {new_name}")
                                    success_count += 1
                                    found = True
                                    break
                    
                    if not found:
                        print(f"❌ No K{specified_k} file found for {prefix}")
    
    print(f"\n📊 Done! Copied {success_count} files to '{output_folder}/'")

# Execute
k_assignments = {'alti': 3, 'grp': 5, 'phylo': 5}
simple_folder_based_copy(k_assignments)

🎯 Copying .meanQ files using folder names...
✅ alti_50_k3.meanQ
✅ alti_95_k3.meanQ
✅ grp_80_k5.meanQ
✅ phylo_90_k5.meanQ
✅ grp_95_k5.meanQ
✅ alti_90_k3.meanQ
✅ phylo_80_k5.meanQ
✅ phylo_95_k5.meanQ
✅ alti_80_k3.meanQ
✅ grp_90_k5.meanQ
✅ grp_50_k5.meanQ
✅ phylo_50_k5.meanQ

📊 Done! Copied 12 files to '3_fastSTRUCTURE/BEST_results/'
